---
output-file: index.html
---

# HAMUX
> **HAMUX** (**H**ierarchical **A**ssociative **M**emory **U**ser e**X**perience) is a Deep Learning framework designed around *energy*. Every architecture built in HAMUX is a global, Lyapunov energy function. HAMUX bridges modern AI architectures and Hopfield Networks.

## What is HAMUX?


<figure>
<img src="https://raw.githubusercontent.com/bhoov/hamux/main/assets/HyperSynapse-fig1.png" alt="HAMUX Overview" width="700"/>
<figcaption style="color:#999">Explaining the "energy fundamentals" of HAMUX (Layers and Synapses, left) using a 4-layer, 3-synapse example HAM (middle) that can be built using the pseudocode on the right. (NOTE: code is not runnable in newer versions of HAMUX as the API has changed).</figcaption>
</figure>

HAMUX defines two fundamental building blocks of energy: the [**🌀neuron layer**](https://bhoov.github.io/hamux/neurons.html) and the [**🤝hypersynapse**](https://bhoov.github.io/hamux/synapses.html) (an abstraction of a pairwise synapse
to include many-body interactions) connected via a
[**hypergraph**](https://en.wikipedia.org/wiki/Hypergraph). 
It is a fully dynamical system, where the “hidden state” $x_i^\ell$ of each layer
$\ell$ (blue squares in the figure below) is an independent variable that
evolves over time. The update rule of each layer is entirely local: neurons evolve deterministically by accumulating "signals" from only the **connected synapses** (i.e., the red circles in the figure below). This is shown in the
following equation:

$$\tau \frac{d x_{i}^{\ell}}{dt} = -\frac{\partial E}{\partial g_i^\ell}$$

where $g_i^\ell$ are the *activations* (i.e., non-linearities) on each
neuron layer $\ell$, described in the section on [Neuron
Layers](#🌀Neuron-Layers). Concretely, we implement the above
differential equation as the following discretized equation (where the
bold ${\mathbf x}_\ell$ is the collection of all elements in layer $\ell$’s
state):

$$ \mathbf{x}_\ell^{(t+1)} = \mathbf{x}_\ell^{(t)} - \frac{dt}{\tau} \nabla_{\mathbf{g}_\ell}E(t)$$

HAMUX handles all the complexity of scaling this fundamental update
equation to many 🌀neurons and 🤝hypersynapses with as minimal overhead as possible. Essentially, HAMUX is a simplified hypergraph library that allows us to modularly compose energy functions. HAMUX makes it easy to:

1.  Inject your data into the associative memory
2.  Perform inference (a.k.a., "Memory Retrieval", "Error correction", or "the forward pass") by **autograd**-computed gradient descent of the energy function!
3. Build complex, powerful networks using arbitrary energy functions. E.g., we can easily build the [Energy Transformer](https://arxiv.org/abs/2302.07253) in this framework using a couple lines of code. See [this tutorial](https://bhoov.github.io/hamux/tutorials/energy_transformer.html) (WIP).


We are continually trying to enrich our [`tutorials`](https://bhoov.github.io/hamux/tutorials/), which are implemented as working Jupyter Notebooks. HAMUX is built on the amazing [JAX](https://github.com/google/jax) and [`equinox`](https://github.com/patrick-kidger/equinox) libraries.

## How to Use

We can build a simple 4 layer HAM architecture using the following code

In [ ]:
import bbhamux as hmx
# from hamux.lagrangians import lagr_identity, lagr_sigmoid, lagr_softmax, lagr_tanh
import jax
import jax.numpy as jnp
import jax.random as jr
import jax.tree_util as jtu
import equinox as eqx

In [ ]:
img_shape = (32,32,3) # E.g., CIFAR images
                           
neurons = {
    "image": hmx.Neurons(hmx.lagr_identity, img_shape),
    "patch": hmx.Neurons(hmx.lagr_tanh, (11,11,16)),
    "label": hmx.Neurons(hmx.lagr_softmax, (10,)),
    "memory": hmx.Neurons(hmx.lagr_softmax, (25,))
}

rng = jr.PRNGKey(0)
k1, k2, k3, rng = jr.split(rng, 4)

synapses = {
    "conv1": hmx.ConvSynapse(k1, 16,3, (3,3), window_strides=(3,3)),
    "dense1": hmx.DenseSynapse(k2, 10, 25),
    "dense2": hmx.DenseSynapse(),
}

connections = [
    (["image","patch"], "conv1"),
    (["label", "memory"], "dense1"),
    (["", ""], "dense2"),
]